# Network analysis in Senegal

### Objectives
    1)	Use measures of road-based accessibility to identify road segments that, if rehabilitated, would improve agricultural market activities in Senegal, including during flood conditions.
    2)	Gain a better understanding of the accessibility, connectivity, and criticality of roads in Senegal in relationship to agricultural origins, processing & transfer sites, and markets.

To this end, the team will develop an accessibility model which measures the travel time from sites of agricultural production to their nearest populated areas, processing centers, and markets. 

### Datasets for analysis
#### ORIGIN
    1) agriculture: MapSPAM 2017. Measuring value and potential value. Values might be reassigned to landcover extents (UMD 2019 30m).
    2) population: WorldPop 2020, UN-adjusted.
    3) settlement extent: GRID3 2020.
#### DESTINATION
    4) markets: derived from WorldPop 2020 and GRID3 2020 urban clusters.
    5) agricultural processing hubs: to be acquired.
#### TRAVEL ROUTE
    6) roads: OpenStreetMap, July 2021.
    7) elevation: 
#### OBSTACLE
    8) flood: FATHOM. 1-in-10, 20, and 50 year flood return periods. 
#### INTERVENTION
    9) upcoming road projects: AGEROUTE interventions separate from the World Bank-financed project
    10) targeted road projects: critical road segments identified by this accessibility model's baseline outputs


### Model design
#### Basic formula: 
    (a) Off-road driving time from origin to closest road node
    +
    (b) Driving time from road node in (a) to a destination (closeness measured by road segments speeds)

#### Model origin & destination (OD) sets:
    A)	Travel time from an area that has agricultural value/potential to the nearest processing hub (if provided).
    B)	Travel time from an area that has agricultural value/potential to the nearest larger settlement, (“larger” settlement identified using a case-appropriate population metric to be determined).
    C)	Travel time from an area that has agricultural value/potential to the nearest market.
    D)	Travel time from all settlements to the nearest market.
    E)	Travel time from larger settlements to the nearest market.

#### Before/after scenarios for each OD set:
    1)	Pre-project, baseline weather: No inclement weather. Road network status as of November 2021.
    2)	Pre-project, flood: 1-in-10, 1-in-20 and 1-in-50 year flood return period. Road network status as of November 2021.
    3)	Post-project, baseline weather: No inclement weather. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).
    4)	Post-project, flood: 1-in-10 year flood return period. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).

#### Notes:
    --Destinations are expected to be proximal to the road network, so no measure is taken between road and destination.
    --All travel times will be assigned to each model variation’s point of origin; the aggregation up to admin areas is possible if desired.
    --Obstacles & interventions modify the road segment speeds. Basic formula is then applied to the modified road network.


### Prep workspace

In [1]:
import os, sys
GISFolder = os.getcwd()
GISFolder

'C:\\Users\\wb527163\\GEO-Cdrive-Grace'

In [2]:
# Note: needed to reinstall rtree due to geopandas import error. Did so in the console. 
# conda install -c conda-forge rtree=0.9.3

In [3]:
# load and filter osm network (step 1)
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import time
sys.path.append(r"C:\Users\wb527163\.conda\envs\geo\GOSTnets-master")
import GOSTnets as gn

In [4]:
import networkx as nx
import osmnx as ox
import numpy as np
import rasterio as rt
import shapely
from shapely.geometry import Point, box
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Point
import peartree

In [5]:
#### Might not use these
import fiona
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [6]:
pth = os.path.join(GISFolder, "SEN-Cdrive") # Personal folder system for running model.
pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive'

In [7]:
out_pth = os.path.join(GISFolder, "SEN-Cdrive\outputs") # For storing intermediate outputs from the model.
out_pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive\\outputs'

In [8]:
team_pth = 'R:\\SEN\\GEO' # This is where the unmodified input data is stored. Finalized outputs also housed here.
team_pth

'R:\\SEN\\GEO'

### Import data

In [9]:
nodes = os.path.join(out_pth, "drive_time_node.csv")
nodes = pd.read_csv(nodes)
nodes.head()

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,node_ID,street_count,x,highway,ref,y,geometry
0,0,358284990,3,-12.323471,NaN,NaN,12.381189,POINT (-12.3234708 12.3811895)
1,1,358285037,3,-12.267195,NaN,NaN,12.534741,POINT (-12.2671955 12.5347411)
2,2,496429117,4,-12.293792,NaN,NaN,12.067253,POINT (-12.2937924 12.067253)
3,3,670104023,3,-12.282907,NaN,NaN,12.411030,POINT (-12.2829074 12.4110304)
4,4,670104201,3,-12.360983,NaN,NaN,12.512258,POINT (-12.3609835 12.5122584)


In [10]:
inV_snap = os.path.join(out_pth, "ag_snap.csv") # V as in village or ville (point of origin)
inV_snap = pd.read_csv(inV_snap)
inV_snap.head()

,Unnamed: 0,Unnamed: 0.1,x,y,ID_lc,geometry,index__spam,ID_spam,val,Unnamed: 0.1.1,lc_count,spam_V,NN,NN_dist
0,0,0,-15.990057,16.693707,1.0,POINT (-15.9900569732419 16.6937073347074),NaN,0.1,1.0,NaN,574130,0.000002,5343951929,14607.791436
1,1,1,-15.989787,16.693707,2.0,POINT (-15.9897874786566 16.6937073347074),NaN,0.1,1.0,NaN,574130,0.000002,5343951929,14628.808767
2,2,2,-15.989518,16.693707,3.0,POINT (-15.9895179840714 16.6937073347074),NaN,0.1,1.0,NaN,574130,0.000002,5343951929,14649.852284
3,3,3,-15.925378,16.693707,4.0,POINT (-15.9253782727816 16.6937073347074),NaN,0.1,1.0,NaN,574130,0.000002,3639403839,14029.850649
4,4,4,-15.925109,16.693707,5.0,POINT (-15.9251087781964 16.6937073347074),NaN,0.1,1.0,NaN,574130,0.000002,3639403839,14018.013147


### Prepare origins and road nodes

In [11]:
geometry = [Point(xy) for xy in zip(nodes.x, nodes.y)]
crs = {'init': 'epsg:4326'} 
nodes = GeoDataFrame(nodes, crs=crs, geometry=geometry) 
nodes.head()

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Unnamed: 0,node_ID,street_count,x,highway,ref,y,geometry
0,0,358284990,3,-12.323471,NaN,NaN,12.381189,POINT (-12.32347 12.38119)
1,1,358285037,3,-12.267195,NaN,NaN,12.534741,POINT (-12.26720 12.53474)
2,2,496429117,4,-12.293792,NaN,NaN,12.067253,POINT (-12.29379 12.06725)
3,3,670104023,3,-12.282907,NaN,NaN,12.411030,POINT (-12.28291 12.41103)
4,4,670104201,3,-12.360983,NaN,NaN,12.512258,POINT (-12.36098 12.51226)


In [11]:
# I was having a lot of trouble creating a geodataframe from the snap csv in the settlements version. 
# Recreating the geometry field resolved it.
inV_snap["geometry"] = inV_snap["geometry"].astype('str')
inV_snap["geometry"] = inV_snap["geometry"].str.strip('POINT ')
inV_snap["geometry"] = inV_snap["geometry"].str.strip('()')
XY = inV_snap["geometry"].str.split(" ", expand=True)
inV_snap["X"] = XY[0]
inV_snap["Y"] = XY[1]
inV_snap['X'] = inV_snap['X'].astype('float')
inV_snap['Y'] = inV_snap['Y'].astype('float')
inV_snap = inV_snap.drop(columns=['geometry'])
inV_snap.head()

,Unnamed: 0,Unnamed_ 0,mgrs_code,type,GlobalID,Shape_Leng,Shape_Area,NN,NN_dist,X,Y
0,0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,7761872879,396.520984,-16.721473,12.348636
1,1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,7761872860,918.665696,-16.716457,12.347887
2,2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,7761872847,952.759215,-16.713855,12.350881
3,3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,7761872847,1923.779554,-16.701275,12.355585
4,4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,7761872847,2181.254121,-16.698774,12.356804


In [ ]:
geometry = [Point(xy) for xy in zip(inV_snap.x, inV_snap.y)]
crs = 'epsg:4326' # If file is not saved as WGS84, make sure to change the CRS after this to match the road nodes.
inV_snap = GeoDataFrame(inV_snap, crs=crs, geometry=geometry) 
inV_snap.head()

In [ ]:
inV_snap = inV_snap.to_crs("EPSG:4326")
nodes = nodes.to_crs("EPSG:4326")
inV_snap.crs == nodes.crs

In [15]:
# Ensure nodes coordinates and unique ID field are in the right data type.
# Most of the rows didn't merge correctly from the nodes layer the first time around because node's NN column was int32, not int64.
nodes['x'] = nodes['x'].astype('float')
nodes['y'] = nodes['y'].astype('float')
nodes['node_ID'] = nodes['node_ID'].astype('int64')
nodes = nodes.drop(columns=['street_count', 'highway', 'ref'])
nodes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 368843 entries, 0 to 368842
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   Unnamed: 0  368843 non-null  int64   
 1   node_ID     368843 non-null  int64   
 2   x           368843 non-null  float64 
 3   y           368843 non-null  float64 
 4   geometry    368843 non-null  geometry
dtypes: float64(2), geometry(1), int64(2)
memory usage: 14.1 MB


In [16]:
inV_snap = inV_snap.drop(columns=['Unnamed_ 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])
inV_snap.head()

,vid,mgrs_code,type,GlobalID,Shape_Leng,Shape_Area,NN,NN_dist,X,Y,geometry
0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,7761872879,396.520984,-16.721473,12.348636,POINT (-16.72147 12.34864)
1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,7761872860,918.665696,-16.716457,12.347887,POINT (-16.71646 12.34789)
2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,7761872847,952.759215,-16.713855,12.350881,POINT (-16.71386 12.35088)
3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,7761872847,1923.779554,-16.701275,12.355585,POINT (-16.70128 12.35559)
4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,7761872847,2181.254121,-16.698774,12.356804,POINT (-16.69877 12.35680)


In [17]:
inV_snap['ID_lc'] = inV_snap['ID_lc'].astype('int64')
inV_snap.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 125886 entries, 0 to 125885
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   vid         125886 non-null  int64   
 1   mgrs_code   125886 non-null  object  
 2   type        125886 non-null  object  
 3   GlobalID    125886 non-null  object  
 4   Shape_Leng  125886 non-null  float64 
 5   Shape_Area  125886 non-null  float64 
 6   NN          125886 non-null  int64   
 7   NN_dist     125886 non-null  float64 
 8   X           125886 non-null  float64 
 9   Y           125886 non-null  float64 
 10  geometry    125886 non-null  geometry
dtypes: float64(5), geometry(1), int64(2), object(3)
memory usage: 10.6+ MB


### Prepare elevation raster

#### Elevation: merge into a single tif

In [19]:
# Read points from shapefile
nodes.index = range(len(nodes))
coords = [(x,y) for x, y in zip(nodes.x, nodes.y)]

# Open the raster and store metadata
src = rt.open('R:/SEN/GEO/Elevation/CWA_senegambia.tif')

# Sample the raster at every point location and store values in DataFrame
nodes['elev'] = [x for x in src.sample(coords)]

nodes.head()

,Unnamed: 0,node_ID,x,y,geometry,elev
0,0,358284990,-12.323471,12.381189,POINT (-12.32347 12.38119),[207.0]
1,1,358285037,-12.267195,12.534741,POINT (-12.26720 12.53474),[134.0]
2,2,496429117,-12.293792,12.067253,POINT (-12.29379 12.06725),[1429.0]
3,3,670104023,-12.282907,12.411030,POINT (-12.28291 12.41103),[204.0]
4,4,670104201,-12.360983,12.512258,POINT (-12.36098 12.51226),[175.0]


In [20]:
# Remove the brackets from elevation variable.
nodes['elev'] = nodes['elev'].astype(str)
nodes['elev'] = nodes['elev'].str.strip('[]')
nodes['elev'] = nodes['elev'].astype('float')
nodes.head()

,Unnamed: 0,node_ID,x,y,geometry,elev
0,0,358284990,-12.323471,12.381189,POINT (-12.32347 12.38119),207.0
1,1,358285037,-12.267195,12.534741,POINT (-12.26720 12.53474),134.0
2,2,496429117,-12.293792,12.067253,POINT (-12.29379 12.06725),1429.0
3,3,670104023,-12.282907,12.411030,POINT (-12.28291 12.41103),204.0
4,4,670104201,-12.360983,12.512258,POINT (-12.36098 12.51226),175.0


In [21]:
# Read points from shapefile
inV_snap.index = range(len(inV_snap))
coords = [(x,y) for x, y in zip(inV_snap.X, inV_snap.Y)]

# Sample the raster at every point location and store values in DataFrame
inV_snap['elev'] = [x for x in src.sample(coords)]
inV_snap

,vid,mgrs_code,type,GlobalID,Shape_Leng,Shape_Area,NN,NN_dist,X,Y,geometry,elev
0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,7761872879,396.520984,-16.721473,12.348636,POINT (-16.72147 12.34864),[13.0]
1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,7761872860,918.665696,-16.716457,12.347887,POINT (-16.71646 12.34789),[12.0]
2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,7761872847,952.759215,-16.713855,12.350881,POINT (-16.71386 12.35088),[7.0]
3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,7761872847,1923.779554,-16.701275,12.355585,POINT (-16.70128 12.35559),[7.0]
4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,7761872847,2181.254121,-16.698774,12.356804,POINT (-16.69877 12.35680),[5.0]
...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,28QED6412_03,hamlet,{5555A010-36B2-47D2-96C4-BDD1E59111ED},0.005397,0.000002,8592243241,5089.086102,-14.397828,16.394143,POINT (-14.39783 16.39414),[24.0]
125882,125882,28QED6413_03,hamlet,{20205A44-8B9D-4FCE-B14C-53826594DB5A},0.003610,0.000001,8592243457,4062.119008,-14.397473,16.405677,POINT (-14.39747 16.40568),[23.0]
125883,125883,28QED6413_04,hamlet,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},0.015471,0.000008,8592243457,4008.851333,-14.400364,16.404303,POINT (-14.40036 16.40430),[24.0]
125884,125884,28QED6424_03,hamlet,{51593C65-B268-4BA1-8212-E43232C021FF},0.003883,0.000001,3646207611,1826.174300,-14.396244,16.497151,POINT (-14.39624 16.49715),[7.0]


In [22]:
inV_snap['elev'] = inV_snap['elev'].astype(str)
inV_snap['elev'] = inV_snap['elev'].str.strip('[]')
inV_snap['elev'] = inV_snap['elev'].astype('float')
inV_snap.head()

,vid,mgrs_code,type,GlobalID,Shape_Leng,Shape_Area,NN,NN_dist,X,Y,geometry,elev
0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,7761872879,396.520984,-16.721473,12.348636,POINT (-16.72147 12.34864),13.0
1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,7761872860,918.665696,-16.716457,12.347887,POINT (-16.71646 12.34789),12.0
2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,7761872847,952.759215,-16.713855,12.350881,POINT (-16.71386 12.35088),7.0
3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,7761872847,1923.779554,-16.701275,12.355585,POINT (-16.70128 12.35559),7.0
4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,7761872847,2181.254121,-16.698774,12.356804,POINT (-16.69877 12.35680),5.0


### Elevation adjusted walk time to road.

In [23]:
#%% If starting new session, reload graph from file
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime.pickle")

#### Next, do the same for origin points.

In [24]:
nodes.rename(columns={'node_ID':'NN'}, inplace=True)
nodes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 368843 entries, 0 to 368842
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   Unnamed: 0  368843 non-null  int64   
 1   NN          368843 non-null  int64   
 2   x           368843 non-null  float64 
 3   y           368843 non-null  float64 
 4   geometry    368843 non-null  geometry
 5   elev        368843 non-null  float64 
dtypes: float64(3), geometry(1), int64(2)
memory usage: 16.9 MB


In [26]:
zvalues = pd.merge(inV_snap, nodes, on='NN', how='left')
zvalues

,vid,mgrs_code,type,GlobalID,Shape_Leng,Shape_Area,NN,NN_dist,X,Y,geometry_x,elev_x,Unnamed: 0,x,y,geometry_y,elev_y
0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,7761872879,396.520984,-16.721473,12.348636,POINT (-16.72147 12.34864),13.0,202442,-16.724557,12.350548,POINT (-16.72456 12.35055),6.0
1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,7761872860,918.665696,-16.716457,12.347887,POINT (-16.71646 12.34789),12.0,202441,-16.722870,12.353291,POINT (-16.72287 12.35329),18.0
2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,7761872847,952.759215,-16.713855,12.350881,POINT (-16.71386 12.35088),7.0,202440,-16.718793,12.357995,POINT (-16.71879 12.35799),14.0
3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,7761872847,1923.779554,-16.701275,12.355585,POINT (-16.70128 12.35559),7.0,202440,-16.718793,12.357995,POINT (-16.71879 12.35799),14.0
4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,7761872847,2181.254121,-16.698774,12.356804,POINT (-16.69877 12.35680),5.0,202440,-16.718793,12.357995,POINT (-16.71879 12.35799),14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,28QED6412_03,hamlet,{5555A010-36B2-47D2-96C4-BDD1E59111ED},0.005397,0.000002,8592243241,5089.086102,-14.397828,16.394143,POINT (-14.39783 16.39414),24.0,47115,-14.424516,16.432258,POINT (-14.42452 16.43226),22.0
125882,125882,28QED6413_03,hamlet,{20205A44-8B9D-4FCE-B14C-53826594DB5A},0.003610,0.000001,8592243457,4062.119008,-14.397473,16.405677,POINT (-14.39747 16.40568),23.0,47199,-14.420648,16.434798,POINT (-14.42065 16.43480),18.0
125883,125883,28QED6413_04,hamlet,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},0.015471,0.000008,8592243457,4008.851333,-14.400364,16.404303,POINT (-14.40036 16.40430),24.0,47199,-14.420648,16.434798,POINT (-14.42065 16.43480),18.0
125884,125884,28QED6424_03,hamlet,{51593C65-B268-4BA1-8212-E43232C021FF},0.003883,0.000001,3646207611,1826.174300,-14.396244,16.497151,POINT (-14.39624 16.49715),7.0,46827,-14.412651,16.492467,POINT (-14.41265 16.49247),12.0


In [27]:
zvalues.rename(columns={'elev_y':'node_elev'}, inplace=True)
zvalues.rename(columns={'elev_x':'v_elev'}, inplace=True)
zvalues.dtypes # Make sure these two columns are float.

vid              int64
mgrs_code       object
type            object
GlobalID        object
Shape_Leng     float64
Shape_Area     float64
NN               int64
NN_dist        float64
X              float64
Y              float64
geometry_x    geometry
v_elev         float64
Unnamed: 0       int64
x              float64
y              float64
geometry_y    geometry
node_elev      float64
dtype: object

In [28]:
# Time is in seconds.
def generate_walktimes(df, start = 'v_elev', end = 'node_elev', dist = 'NN_dist', max_walkspeed = 6, min_speed = 0.1):
    # Irmischer & Clarke hiking function
    def speed(incline_ratio, max_speed):
        walkspeed = (0.95 * (0.11 + np.exp(-(np.square(incline_ratio + 0.05) / np.square(2*30))))) * 3.6
        return walkspeed

    speeds = {}
    times = {}

    for index, data in df.iterrows():
        if data[dist] > 0:
            delta_elevation = data[end] - data[start]
            incline_ratio = delta_elevation / data[dist]
            speed_kmph = speed(incline_ratio = incline_ratio, max_speed = max_walkspeed)
            speed_kmph = max(speed_kmph, min_speed)
            speeds[index] = (speed_kmph)
            times[index] = (data[dist] / 1000 * 3600 / speed_kmph)

    speed_df = pd.DataFrame.from_dict(speeds, orient = 'index')
    time_df = pd.DataFrame.from_dict(times, orient = 'index')

    df['walkspeed'] = speed_df[0]
    df['walk_time'] = time_df[0]
    
    return df

In [30]:
zwalk = generate_walktimes(zvalues) # Took a couple seconds.
zwalk

,vid,mgrs_code,type,GlobalID,Shape_Leng,Shape_Area,NN,NN_dist,X,Y,geometry_x,v_elev,Unnamed: 0,x,y,geometry_y,node_elev,walkspeed,walk_time
0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,7761872879,396.520984,-16.721473,12.348636,POINT (-16.72147 12.34864),13.0,202442,-16.724557,12.350548,POINT (-16.72456 12.35055),6.0,3.796199,376.027584
1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,7761872860,918.665696,-16.716457,12.347887,POINT (-16.71646 12.34789),12.0,202441,-16.722870,12.353291,POINT (-16.72287 12.35329),18.0,3.796197,871.186753
2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,7761872847,952.759215,-16.713855,12.350881,POINT (-16.71386 12.35088),7.0,202440,-16.718793,12.357995,POINT (-16.71879 12.35799),14.0,3.796197,903.518254
3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,7761872847,1923.779554,-16.701275,12.355585,POINT (-16.70128 12.35559),7.0,202440,-16.718793,12.357995,POINT (-16.71879 12.35799),14.0,3.796197,1824.353665
4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,7761872847,2181.254121,-16.698774,12.356804,POINT (-16.69877 12.35680),5.0,202440,-16.718793,12.357995,POINT (-16.71879 12.35799),14.0,3.796197,2068.521309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,28QED6412_03,hamlet,{5555A010-36B2-47D2-96C4-BDD1E59111ED},0.005397,0.000002,8592243241,5089.086102,-14.397828,16.394143,POINT (-14.39783 16.39414),24.0,47115,-14.424516,16.432258,POINT (-14.42452 16.43226),22.0,3.796198,4826.068502
125882,125882,28QED6413_03,hamlet,{20205A44-8B9D-4FCE-B14C-53826594DB5A},0.003610,0.000001,8592243457,4062.119008,-14.397473,16.405677,POINT (-14.39747 16.40568),23.0,47199,-14.420648,16.434798,POINT (-14.42065 16.43480),18.0,3.796198,3852.177739
125883,125883,28QED6413_04,hamlet,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},0.015471,0.000008,8592243457,4008.851333,-14.400364,16.404303,POINT (-14.40036 16.40430),24.0,47199,-14.420648,16.434798,POINT (-14.42065 16.43480),18.0,3.796198,3801.663057
125884,125884,28QED6424_03,hamlet,{51593C65-B268-4BA1-8212-E43232C021FF},0.003883,0.000001,3646207611,1826.174300,-14.396244,16.497151,POINT (-14.39624 16.49715),7.0,46827,-14.412651,16.492467,POINT (-14.41265 16.49247),12.0,3.796197,1731.792860


In [31]:
zwalk.isna().sum() # Make sure the rows merged correctly. If they didn't, double check the dtypes on NN to make sure they were the same in the nodes and snap dataframes.

vid           0
mgrs_code     0
type          0
GlobalID      0
Shape_Leng    0
Shape_Area    0
NN            0
NN_dist       0
X             0
Y             0
geometry_x    0
v_elev        0
Unnamed: 0    0
x             0
y             0
geometry_y    0
node_elev     0
walkspeed     0
walk_time     0
dtype: int64

In [32]:
zwalk.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   vid         125886 non-null  int64   
 1   mgrs_code   125886 non-null  object  
 2   type        125886 non-null  object  
 3   GlobalID    125886 non-null  object  
 4   Shape_Leng  125886 non-null  float64 
 5   Shape_Area  125886 non-null  float64 
 6   NN          125886 non-null  int64   
 7   NN_dist     125886 non-null  float64 
 8   X           125886 non-null  float64 
 9   Y           125886 non-null  float64 
 10  geometry_x  125886 non-null  geometry
 11  v_elev      125886 non-null  float64 
 12  Unnamed: 0  125886 non-null  int64   
 13  x           125886 non-null  float64 
 14  y           125886 non-null  float64 
 15  geometry_y  125886 non-null  geometry
 16  node_elev   125886 non-null  float64 
 17  walkspeed   125886 non-null  float64 
 18  walk_time   1258

In [36]:
zwalk.to_csv('SEN-Cdrive/outputs/fromagriculture/zwalk_intermediatefile.csv', index=True)

In [34]:
# Clean up the file for only the necessary columns.
zwalk2 = zwalk.drop(columns=['mgrs_code','type', 'Shape_Leng', 'Shape_Area', 'Unnamed: 0'])
zwalk2.rename(columns={'X':'X_haml'}, inplace=True)
zwalk2.rename(columns={'Y':'Y_haml'}, inplace=True)
zwalk2.rename(columns={'geometry_x':'geom_haml'}, inplace=True)
zwalk2.rename(columns={'x':'X_road'}, inplace=True)
zwalk2.rename(columns={'y':'Y_road'}, inplace=True)
zwalk2.rename(columns={'geometry_y':'geom_road'}, inplace=True)
zwalk2.dtypes

vid             int64
GlobalID       object
NN              int64
NN_dist       float64
X_haml        float64
Y_haml        float64
geom_haml    geometry
v_elev        float64
X_road        float64
Y_road        float64
geom_road    geometry
node_elev     float64
walkspeed     float64
walk_time     float64
dtype: object

In [35]:
# Convert from seconds to minutes
zwalk2['walk_time'] = zwalk2['walk_time'] / 60
zwalk2

,vid,GlobalID,NN,NN_dist,X_haml,Y_haml,geom_haml,v_elev,X_road,Y_road,geom_road,node_elev,walkspeed,walk_time
0,0,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},7761872879,396.520984,-16.721473,12.348636,POINT (-16.72147 12.34864),13.0,-16.724557,12.350548,POINT (-16.72456 12.35055),6.0,3.796199,6.267126
1,1,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},7761872860,918.665696,-16.716457,12.347887,POINT (-16.71646 12.34789),12.0,-16.722870,12.353291,POINT (-16.72287 12.35329),18.0,3.796197,14.519779
2,2,{D03C2B85-5F35-4EE8-8346-B83494628F26},7761872847,952.759215,-16.713855,12.350881,POINT (-16.71386 12.35088),7.0,-16.718793,12.357995,POINT (-16.71879 12.35799),14.0,3.796197,15.058638
3,3,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},7761872847,1923.779554,-16.701275,12.355585,POINT (-16.70128 12.35559),7.0,-16.718793,12.357995,POINT (-16.71879 12.35799),14.0,3.796197,30.405894
4,4,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},7761872847,2181.254121,-16.698774,12.356804,POINT (-16.69877 12.35680),5.0,-16.718793,12.357995,POINT (-16.71879 12.35799),14.0,3.796197,34.475355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,{5555A010-36B2-47D2-96C4-BDD1E59111ED},8592243241,5089.086102,-14.397828,16.394143,POINT (-14.39783 16.39414),24.0,-14.424516,16.432258,POINT (-14.42452 16.43226),22.0,3.796198,80.434475
125882,125882,{20205A44-8B9D-4FCE-B14C-53826594DB5A},8592243457,4062.119008,-14.397473,16.405677,POINT (-14.39747 16.40568),23.0,-14.420648,16.434798,POINT (-14.42065 16.43480),18.0,3.796198,64.202962
125883,125883,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},8592243457,4008.851333,-14.400364,16.404303,POINT (-14.40036 16.40430),24.0,-14.420648,16.434798,POINT (-14.42065 16.43480),18.0,3.796198,63.361051
125884,125884,{51593C65-B268-4BA1-8212-E43232C021FF},3646207611,1826.174300,-14.396244,16.497151,POINT (-14.39624 16.49715),7.0,-14.412651,16.492467,POINT (-14.41265 16.49247),12.0,3.796197,28.863214


In [37]:
# Save to disk.
zwalk2.to_csv('SEN-Cdrive/outputs/fromagriculture/zwalk.csv', index=True)

### Create multi-modal travel times by combining walk time to road with drive time to nth nearest service.

In [8]:
zwalk2 = os.path.join(out_pth, "fromagriculture/zwalk.csv") 
zwalk2 = pd.read_csv(zwalk2)
zwalk2

,Unnamed: 0,vid,GlobalID,NN,NN_dist,X_haml,Y_haml,geom_haml,v_elev,X_road,Y_road,geom_road,node_elev,walkspeed,walk_time
0,0,0,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},7761872879,396.520984,-16.721473,12.348636,POINT (-16.721473282454415 12.348636090165247),13.0,-16.724557,12.350548,POINT (-16.7245571 12.3505478),6.0,3.796199,6.267126
1,1,1,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},7761872860,918.665696,-16.716457,12.347887,POINT (-16.716456507935607 12.34788723564788),12.0,-16.722870,12.353291,POINT (-16.7228695 12.3532914),18.0,3.796197,14.519779
2,2,2,{D03C2B85-5F35-4EE8-8346-B83494628F26},7761872847,952.759215,-16.713855,12.350881,POINT (-16.713855008830738 12.350880992129111),7.0,-16.718793,12.357995,POINT (-16.7187927 12.3579947),14.0,3.796197,15.058638
3,3,3,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},7761872847,1923.779554,-16.701275,12.355585,POINT (-16.701275174874546 12.355585269999269),7.0,-16.718793,12.357995,POINT (-16.7187927 12.3579947),14.0,3.796197,30.405894
4,4,4,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},7761872847,2181.254121,-16.698774,12.356804,POINT (-16.698773736706396 12.356804484409668),5.0,-16.718793,12.357995,POINT (-16.7187927 12.3579947),14.0,3.796197,34.475355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,125881,{5555A010-36B2-47D2-96C4-BDD1E59111ED},8592243241,5089.086102,-14.397828,16.394143,POINT (-14.397827933065358 16.394142941310925),24.0,-14.424516,16.432258,POINT (-14.4245157 16.4322581),22.0,3.796198,80.434475
125882,125882,125882,{20205A44-8B9D-4FCE-B14C-53826594DB5A},8592243457,4062.119008,-14.397473,16.405677,POINT (-14.397473236932905 16.405676938062538),23.0,-14.420648,16.434798,POINT (-14.4206477 16.4347983),18.0,3.796198,64.202962
125883,125883,125883,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},8592243457,4008.851333,-14.400364,16.404303,POINT (-14.400364192239715 16.404303134272737),24.0,-14.420648,16.434798,POINT (-14.4206477 16.4347983),18.0,3.796198,63.361051
125884,125884,125884,{51593C65-B268-4BA1-8212-E43232C021FF},3646207611,1826.174300,-14.396244,16.497151,POINT (-14.396243884215123 16.49715066208162),7.0,-14.412651,16.492467,POINT (-14.412651 16.4924672),12.0,3.796197,28.863214


In [9]:
HDurban_all = os.path.join(out_pth, "fromagriculture/HDurban_all.csv") 
HDurban_all = pd.read_csv(HDurban_all)
HDurban_all

,Unnamed: 0,NN,HDurban1,HDurban2,HDurban3
0,0,7761872879,62.713982,66.144367,199.650409
1,1,7761872860,61.991537,65.421922,198.927964
2,2,7761872847,60.942482,64.372867,197.878909
3,3,8604836683,68.078912,69.841801,195.450304
4,4,8604829828,73.671735,75.434624,201.043127
...,...,...,...,...,...
23103,23103,8449983186,60.225974,61.707741,67.705341
23104,23104,4072935008,86.344963,88.583453,103.728980
23105,23105,4072935012,86.912227,89.150717,104.296245
23106,23106,8558304561,103.968632,106.207122,121.352649


In [10]:
HDurban_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23108 entries, 0 to 23107
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  23108 non-null  int64  
 1   NN          23108 non-null  int64  
 2   HDurban1    23106 non-null  float64
 3   HDurban2    23108 non-null  float64
 4   HDurban3    23108 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 902.8 KB


In [11]:
zwalk2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125886 entries, 0 to 125885
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  125886 non-null  int64  
 1   vid         125886 non-null  int64  
 2   GlobalID    125886 non-null  object 
 3   NN          125886 non-null  int64  
 4   NN_dist     125886 non-null  float64
 5   X_haml      125886 non-null  float64
 6   Y_haml      125886 non-null  float64
 7   geom_haml   125886 non-null  object 
 8   v_elev      125886 non-null  float64
 9   X_road      125886 non-null  float64
 10  Y_road      125886 non-null  float64
 11  geom_road   125886 non-null  object 
 12  node_elev   125886 non-null  float64
 13  walkspeed   125886 non-null  float64
 14  walk_time   125886 non-null  float64
dtypes: float64(9), int64(3), object(3)
memory usage: 14.4+ MB


In [12]:
zwalk_HDurban = zwalk2.merge(HDurban_all, how='left', left_on='NN', right_on='NN', sort=False)
zwalk_HDurban.tail(30)

,Unnamed: 0_x,vid,GlobalID,NN,NN_dist,X_haml,Y_haml,geom_haml,v_elev,X_road,Y_road,geom_road,node_elev,walkspeed,walk_time,Unnamed: 0_y,HDurban1,HDurban2,HDurban3
125856,125856,125856,{22551739-F746-4C42-9CFA-918DA861DAE8},9041943742,3728.065735,-13.634542,15.748839,POINT (-13.634542240843889 15.748839239773597),76.0,-13.658344,15.773416,POINT (-13.6583444 15.7734165),51.0,3.796198,58.923147,21423,271.574996,273.084994,275.002987
125857,125857,125857,{A016A46D-7DEB-4516-85C3-D734CE32BB08},1987817404,6756.790720,-13.632385,15.908039,POINT (-13.632384905651481 15.908038898553485),72.0,-13.577587,15.877758,POINT (-13.5775875 15.8777581),47.0,3.796198,106.793020,21687,251.001380,252.511378,254.429371
125858,125858,125858,{0B17D0D9-24E9-45D6-A040-CE0BCAE1EBC5},1987817404,6783.427144,-13.631616,15.909771,POINT (-13.631615806842289 15.909770584818125),72.0,-13.577587,15.877758,POINT (-13.5775875 15.8777581),47.0,3.796198,107.214016,21687,251.001380,252.511378,254.429371
125859,125859,125859,{7BC3CDC3-F0F8-4864-A337-F26AE75A09EB},3650030910,3534.224026,-13.629271,16.074080,POINT (-13.629271016877984 16.074080435238983),12.0,-13.624898,16.105741,POINT (-13.6248982 16.1057413),17.0,3.796197,55.859434,21503,265.165442,266.647209,272.644808
125860,125860,125860,{34ED2616-7D64-45E9-8316-14E18D96BB3F},3650030910,3335.205126,-13.628169,16.075765,POINT (-13.628169393420567 16.075764507281782),13.0,-13.624898,16.105741,POINT (-13.6248982 16.1057413),17.0,3.796198,52.713882,21503,265.165442,266.647209,272.644808
125861,125861,125861,{6F416481-5D08-4A91-8986-33373551EA6F},9207762351,2928.681591,-14.312926,16.129799,POINT (-14.312925975367385 16.129798701531637),40.0,-14.326172,16.106627,POINT (-14.3261719 16.1066268),46.0,3.796197,46.288661,21534,238.731555,240.213322,246.210922
125862,125862,125862,{CA24AA7D-190F-42AF-9E2B-B4B99417510D},9207762351,2994.402598,-14.310434,16.129016,POINT (-14.310434450218114 16.1290157817221),40.0,-14.326172,16.106627,POINT (-14.3261719 16.1066268),46.0,3.796197,47.327400,21534,238.731555,240.213322,246.210922
125863,125863,125863,{E5BA361E-3599-4418-8957-1307399E23CB},9207762257,3222.986418,-14.285924,16.128193,POINT (-14.285923703058417 16.128193206058107),43.0,-14.268569,16.104374,POINT (-14.2685689 16.1043737),43.0,3.796198,50.940231,21533,230.756438,232.238205,238.235805
125864,125864,125864,{CFAB5FBB-E2E7-4E9B-85C0-2188C61DB227},6128872306,6138.643077,-14.133517,16.126855,POINT (-14.133517144961344 16.126854804761635),38.0,-14.106423,16.175776,POINT (-14.1064229 16.1757757),30.0,3.796198,97.023024,21531,224.681084,226.162851,232.160451
125865,125865,125865,{2A8F9A50-0609-48EF-A025-F4AC1D8860E6},6457408822,6651.716250,-14.022600,16.124635,POINT (-14.022600038102192 16.12463463543429),31.0,-13.963910,16.104731,POINT (-13.96391 16.1047312),28.0,3.796198,105.132296,21527,230.553613,232.035380,238.032980


In [13]:
zwalk_HDurban["mm1HDurban"] = 0
zwalk_HDurban["mm1HDurban"] = zwalk_HDurban["walk_time"] + zwalk_HDurban["HDurban1"]
zwalk_HDurban["mm2HDurban"] = 0
zwalk_HDurban["mm2HDurban"] = zwalk_HDurban["walk_time"] + zwalk_HDurban["HDurban2"]
zwalk_HDurban["mm3HDurban"] = 0
zwalk_HDurban["mm3HDurban"] = zwalk_HDurban["walk_time"] + zwalk_HDurban["HDurban3"]
zwalk_HDurban

,Unnamed: 0_x,vid,GlobalID,NN,NN_dist,X_haml,Y_haml,geom_haml,v_elev,X_road,...,node_elev,walkspeed,walk_time,Unnamed: 0_y,HDurban1,HDurban2,HDurban3,mm1HDurban,mm2HDurban,mm3HDurban
0,0,0,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},7761872879,396.520984,-16.721473,12.348636,POINT (-16.721473282454415 12.348636090165247),13.0,-16.724557,...,6.0,3.796199,6.267126,0,62.713982,66.144367,199.650409,68.981109,72.411493,205.917536
1,1,1,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},7761872860,918.665696,-16.716457,12.347887,POINT (-16.716456507935607 12.34788723564788),12.0,-16.722870,...,18.0,3.796197,14.519779,1,61.991537,65.421922,198.927964,76.511316,79.941701,213.447743
2,2,2,{D03C2B85-5F35-4EE8-8346-B83494628F26},7761872847,952.759215,-16.713855,12.350881,POINT (-16.713855008830738 12.350880992129111),7.0,-16.718793,...,14.0,3.796197,15.058638,2,60.942482,64.372867,197.878909,76.001120,79.431504,212.937547
3,3,3,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},7761872847,1923.779554,-16.701275,12.355585,POINT (-16.701275174874546 12.355585269999269),7.0,-16.718793,...,14.0,3.796197,30.405894,2,60.942482,64.372867,197.878909,91.348377,94.778761,228.284804
4,4,4,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},7761872847,2181.254121,-16.698774,12.356804,POINT (-16.698773736706396 12.356804484409668),5.0,-16.718793,...,14.0,3.796197,34.475355,2,60.942482,64.372867,197.878909,95.417837,98.848222,232.354264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,125881,{5555A010-36B2-47D2-96C4-BDD1E59111ED},8592243241,5089.086102,-14.397828,16.394143,POINT (-14.397827933065358 16.394142941310925),24.0,-14.424516,...,22.0,3.796198,80.434475,22763,184.980367,186.462134,192.459734,265.414842,266.896609,272.894209
125882,125882,125882,{20205A44-8B9D-4FCE-B14C-53826594DB5A},8592243457,4062.119008,-14.397473,16.405677,POINT (-14.397473236932905 16.405676938062538),23.0,-14.420648,...,18.0,3.796198,64.202962,22774,184.681609,186.163376,192.160976,248.884571,250.366338,256.363938
125883,125883,125883,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},8592243457,4008.851333,-14.400364,16.404303,POINT (-14.400364192239715 16.404303134272737),24.0,-14.420648,...,18.0,3.796198,63.361051,22774,184.681609,186.163376,192.160976,248.042660,249.524427,255.522026
125884,125884,125884,{51593C65-B268-4BA1-8212-E43232C021FF},3646207611,1826.174300,-14.396244,16.497151,POINT (-14.396243884215123 16.49715066208162),7.0,-14.412651,...,12.0,3.796197,28.863214,22809,180.888369,182.370136,188.367736,209.751583,211.233350,217.230950


In [14]:
zwalk_HDurban.to_csv(os.path.join(out_pth, 'fromagriculture/fromagriculture_multimodal_HDurban.csv'))

### End of script. Load into QGIS or Arc and visualize at 10 min intervals. 
QML file for symbology in QGIS:
R:\GEOGlobal\Design\symb_traveltimes_10min.qml